In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome as wdChrome, Firefox as wdFirefox, Edge as wdEdge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as selExcep 

from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

import time

In [2]:
home_url = 'https://www.google.com/maps/place/Lahore,+Punjab,+Pakistan/@31.4831569,74.1943055,11z/data=!3m1!4b1!4m5!3m4!1s0x39190483e58107d9:0xc23abe6ccc7e2462!8m2!3d31.5203696!4d74.3587473'
driver_type = "chrome"
service = ChromeService(executable_path = ChromeDriverManager().install())
driver = wdChrome(service= service)
driver.get(home_url)
action = ActionChains(driver)
time.sleep(1)

In [3]:
MAP_SEARCH_XPATH = '//*[@id="searchboxinput"]'
LOCATION_BOX_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'

LOC_BOX_CLASS = "hfpxzc"

In [4]:
def search_map(query: str) -> None:
    map_searchbar = driver.find_element(By.XPATH, MAP_SEARCH_XPATH)
    map_searchbar.clear()
    map_searchbar.send_keys(query)
    map_searchbar.send_keys(Keys.RETURN)

In [5]:
search_map("Businesses")
time.sleep(5)

In [6]:
loc_box = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
action.move_to_element(loc_box).perform()

In [8]:
loc_div = driver.find_element(By.XPATH, LOCATION_BOX_XPATH)
action.move_to_element(loc_div).perform()
hrefs_set = set()


last_href = None
find_elem = False
while not find_elem:
    
    loc_boxes = driver.find_elements(By.CLASS_NAME, LOC_BOX_CLASS)
    hrefs = [loc_box.get_attribute("href") for loc_box in loc_boxes]
    hrefs_set.update(hrefs)
    loc_div.send_keys(Keys.END)
    time.sleep(2)
    
    if(last_href == None):
        last_href = hrefs[-1]
        continue
    
    if(last_href == hrefs[-1]):
        find_elem = True
    else:
        last_href = hrefs[-1]

In [15]:
hrefs_set

{'https://www.google.com/maps/place/2+Zee+Business+Solution/data=!4m7!3m6!1s0x391904545ae7e845:0xea085caa9523dbe!8m2!3d31.5163115!4d74.3551786!16s%2Fg%2F11cmqpfp0r!19sChIJRejnWlQEGTkRvj1SqcqFoA4?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/2+Zee+Business+Solutions+(ezeeprintpk)/data=!4m7!3m6!1s0x391903c4966f1167:0x9ebf096315ab99e0!8m2!3d31.4797325!4d74.3011091!16s%2Fg%2F11fxvxt8qt!19sChIJZxFvlsQDGTkR4JmrFWMJv54?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/3-E+Business+Solutions/data=!4m7!3m6!1s0x391904070bc4f903:0x3b37ce2871f07e90!8m2!3d31.4783011!4d74.3134868!16s%2Fg%2F1x5f8rhn!19sChIJA_nECwcEGTkRkH7wcSjONzs?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/A+1+BUSINESS+MASTER/data=!4m7!3m6!1s0x3919012148f54725:0xfb6883b54e116846!8m2!3d31.4541046!4d74.3095786!16s%2Fg%2F11ktmtb71d!19sChIJJUf1SCEBGTkRRmgRTrWDaPs?authuser=0&hl=en&rclk=1',
 'https://www.google.com/maps/place/ABS+(Acme+Business+Services)/data=!4m7!3m6!1s0x3919015dfdc25dd1:0xae7

In [17]:
test_href = hrefs_set.pop()

In [18]:
test_href

'https://www.google.com/maps/place/The+Business+Hikes/data=!4m7!3m6!1s0x39190396847f5001:0x1b4df431bc117668!8m2!3d31.5070643!4d74.2785718!16s%2Fg%2F11rgght4yp!19sChIJAVB_hJYDGTkRaHYRvDH0TRs?authuser=0&hl=en&rclk=1'

In [19]:
test_href.split("!")

['https://www.google.com/maps/place/The+Business+Hikes/data=',
 '4m7',
 '3m6',
 '1s0x39190396847f5001:0x1b4df431bc117668',
 '8m2',
 '3d31.5070643',
 '4d74.2785718',
 '16s%2Fg%2F11rgght4yp',
 '19sChIJAVB_hJYDGTkRaHYRvDH0TRs?authuser=0&hl=en&rclk=1']

In [34]:
test_href2 = hrefs_set.pop()

In [42]:
test_href2 = "https://www.google.com/maps/place/Lahore+Central+Business+District+Development+Authority/data=!4m7!3m6!1s0x39190520d7e79e63:0xcb907103dff63ca8!8m2!3d31.5080263!4d74.3263346!16s%2Fg%2F11rcvcfhpl!19sChIJY57n1yAFGTkRqDz23wNxkMs?authuser=0&hl=en&rclk=1"

In [44]:
test_href2 = test_href2.split("!")

In [45]:
test_href2

['https://www.google.com/maps/place/Lahore+Central+Business+District+Development+Authority/data=',
 '4m7',
 '3m6',
 '1s0x39190520d7e79e63:0xcb907103dff63ca8',
 '8m2',
 '3d31.5080263',
 '4d74.3263346',
 '16s%2Fg%2F11rcvcfhpl',
 '19sChIJY57n1yAFGTkRqDz23wNxkMs?authuser=0&hl=en&rclk=1']